In [1]:
import sys
import os
import shutil
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import healpix as hp

import easygems
import easygems.remap as egr
import easygems.healpix as egh

import dask
import dask.array as da

import zarr


In [6]:

def get_mpas_lonlat(ds, lonname, latname, degrees=True, negative=True, verbose=False, wraparound=True):
    '''Get latitude and longitude from MPAS "static" file,
       convert to degrees (default),
       convert to [-180, 180] convention (default)

    ds : xr.Dataset
        data set that needs to have lat and lon values
    latname : str
        name of the latitude variable
    lonname : str
        name of the longitude variable
    degrees : bool
        if true, convert to degrees (ASSUMES RADIANS)
    negative : bool
        if true, convert to -180 format if needed
        if false, convert to 360 format if needed
        Assumes unit is degrees, and the conversion is based on minimum longitude value being < 0 or maximum > 180
        Does not "roll" the coordinate (i.e. change the order of the longitudes)
    verbose : bool
        if true print stuff
    '''
    lonrad = ds[lonname]
    latrad = ds[latname]
    if verbose:
        print(f"Sizes: {lonrad.shape = }, {latrad.shape = } -- Compare with {ds['nCells'].shape}")
        print(f"[initial] Lat min/max: {latrad.min().item()}, {latrad.max().item()}, Lon min/max: {lonrad.min().item()},{lonrad.max().item()}")
    
    if degrees:
        # lon and lat are in radians
        lon = np.rad2deg(lonrad) 
        lat = np.rad2deg(latrad)
    else:
        lon = lonrad
        lat = latrad

    if verbose:
        print(f"[degrees] Lat min/max: {lat.min().item()}, {lat.max().item()}, Lon min/max: {lon.min().item()},{lon.max().item()}")

    if negative:
        if lon.max().item() >= 180:
            lon=(lon + 180) % 360 - 180  # [-180, 180)
    else:
        if lon.min().item() < 0:
            lon += 180
    result = (lon, lat)
    if wraparound:
        orig_size = len(lon)
        lon_wrap = xr.concat([lon, lon], dim=lon.dims[0])
        lon_wrap[orig_size:] = lon+360
        orig_size = len(lat)
        lat_wrap = xr.concat([lat, lat], dim=lat.dims[0])
        if negative:
            if lon_wrap.max().item() >= 180:
                lon_wrap=(lon_wrap + 180) % 720 - 180
        else:
            if lon.min().item() < 0:
                lon += 180
        result = (lon, lat, lon_wrap, lat_wrap)            

    if verbose:
        print(f"[final] Lat min/max: {lat.min().item()}, {lat.max().item()}, Lon min/max: {lon.min().item()},{lon.max().item()}")
    return result


In [7]:
# mesh description (maybe)
meshloc = Path("/glade/campaign/mmm/wmr/skamaroc/NSC_2023")
meshfil = meshloc / "x1.41943042.static.nc"

# cell-center
ds_static = xr.open_dataset(meshfil)
lon, lat, lon_wrap, lat_wrap = get_mpas_lonlat(ds_static, 'lonCell', 'latCell', degrees=True, negative=True, verbose=True, wraparound=True)
vlon, vlat, vlon_wrap, vlat_wrap = get_mpas_lonlat(ds_static, 'lonVertex', 'latVertex', degrees=True, negative=True, verbose=True, wraparound=True)


Sizes: lonrad.shape = (41943042,), latrad.shape = (41943042,) -- Compare with (41943042,)
[initial] Lat min/max: -1.5707963705062866, 1.5707963705062866, Lon min/max: 0.0,6.2831854820251465
[degrees] Lat min/max: -90.0, 90.0, Lon min/max: 0.0,360.0
[final] Lat min/max: -90.0, 90.0, Lon min/max: -180.0,179.98284912109375
Sizes: lonrad.shape = (83886080,), latrad.shape = (83886080,) -- Compare with (41943042,)
[initial] Lat min/max: -1.5704516172409058, 1.5704516172409058, Lon min/max: 0.0,6.2831854820251465
[degrees] Lat min/max: -89.9802474975586, 89.9802474975586, Lon min/max: 0.0,360.0
[final] Lat min/max: -89.9802474975586, 89.9802474975586, Lon min/max: -180.0,179.98941040039062


<xarray.DataArray 'lonCell' ()> Size: 8B
array(539.98284912)